<a href="https://colab.research.google.com/github/tunatone0111/2020-summer-research/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project
Only runs for python3


## Dependencies

In [1]:
!pip install tensorflow-gpu==2.3.0

     |████████████████████████████████| 320.4MB 42kB/s 


In [2]:
!pip install tensorflow_probability==0.11.0

In [3]:
!pip install attr

  Created wheel for attr: filename=attr-0.3.1-cp36-none-any.whl size=2459 sha256=1e245e114d2815081a323261f8ccf9890c1a82b2f0f5d0cd83c396dff89ce7ff
  Stored in directory: /root/.cache/pip/wheels/f0/96/9b/1f8892a707d17095b5a6eab0275da9d39e68e03a26aee2e726
Successfully built attr


In [7]:
import attr
import functools
import collections
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

## Utils

### for variational layer

In [9]:
tfd = tfp.distributions


def tfp_layer_with_scaled_kl(layer_builder, num_train_examples):
    def scaled_kl_fn(q, p, _):
        return tfd.kl_divergence(q, p) / num_train_examples

    return functools.partial(layer_builder, kernel_divergence_fn=scaled_kl_fn)


def _posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    """Posterior function for variational layer."""
    n = kernel_size + bias_size
    c = np.log(np.expm1(1e-5))
    variable_layer = tfp.layers.VariableLayer(
        2 * n, dtype=dtype,
        initializer=tfp.layers.BlockwiseInitializer([
            keras.initializers.TruncatedNormal(mean=0., stddev=.05, seed=None),
            keras.initializers.Constant(np.log(np.expm1(1e-5)))], sizes=[n, n]))

    def distribution_fn(t):
        scale = 1e-5 + tf.nn.softplus(c + t[Ellipsis, n:])
        return tfd.Independent(tfd.Normal(loc=t[Ellipsis, :n], scale=scale),
                               reinterpreted_batch_ndims=1)
    distribution_layer = tfp.layers.DistributionLambda(distribution_fn)
    return tf.keras.Sequential([variable_layer, distribution_layer])


def _make_prior_fn(kernel_size, bias_size=0, dtype=None):
    del dtype  # TODO(yovadia): Figure out what to do with this.
    loc = tf.zeros(kernel_size + bias_size)

    def distribution_fn(_):
        return tfd.Independent(tfd.Normal(loc=loc, scale=1),
                               reinterpreted_batch_ndims=1)
    return distribution_fn

### layer builder

In [ ]:
def get_layer_builders(method, num_train_examples):
  """Get method-appropriate functions for building and/or applying Keras layers.

  Args:
    method: UQ method (vanilla, svi).
    num_train_examples: Number of training examples. Used to scale KL loss.
  Returns:
    conv2d, dense_layer
  """
    if method == 'vanilla':
        return keras.layers.Conv2D, keras.layers.Dense
    
    tfpl = tfp.layers

    conv2d_variational = tfp_layer_with_scaled_kl(tfpl.Convolution2DFlipout,
                                                  num_train_examples)
    # Only DenseVariational works in v2 / eager mode.
    # FMI: https://github.com/tensorflow/probability/issues/409
    if tf.executing_eagerly():
        def dense_variational(units, activation):
            return tfpl.DenseVariational(
                units,
                make_posterior_fn=_posterior_mean_field,
                make_prior_fn=_make_prior_fn,
                activation=activation,
                kl_weight=1./num_train_examples)
    else:
        dense_variational = tfp_layer_with_scaled_kl(tfpl.DenseFlipout,
                                                     num_train_examples)

    return conv2d_variational, dense_variational

## Model

In [ ]:
class ModelOptions(object):
    """Parameters for model construction and fitting."""
    train_epochs = attr.ib()
    num_train_examples = attr.ib()
    batch_size = attr.ib()
    learning_rate = attr.ib()
    method = attr.ib()
    num_examples_for_predict = attr.ib()
    predictions_per_example = attr.ib()
    input_shape = attr.ib()
    num_classes = attr.ib()


def build_model(opts):
  """Builds a VGGNet Keras model."""
  layer_builders = get_layer_builders(opts.method, opts.num_train_examples)
  conv2d, dense_layer = layer_builders

  inputs = keras.layers.Input(opts.input_shape)
  net = inputs
  logits = dense_last(opts.num_classes)(net)
  return keras.Model(inputs=inputs, outputs=logits)


def build_and_train(opts, dataset_train, dataset_eval, output_dir):
  """Returns a trained MNIST model and saves it to output_dir.

  Args:
    opts: ModelOptions
    dataset_train: Pair of images, labels np.ndarrays for training.
    dataset_eval: Pair of images, labels np.ndarrays for continuous eval.
    output_dir: Directory for the saved model and tensorboard events.
  Returns:
    Trained Keras model.
  """
  model = build_model(opts)
  model.compile(
      keras.optimizers.Adam(opts.learning_rate),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'],
  )

  tensorboard_cb = keras.callbacks.TensorBoard(
      log_dir=output_dir, write_graph=False)

  train_images, train_labels = dataset_train
  assert len(train_images) == opts.num_train_examples, (
      '%d != %d' % (len(train_images), opts.num_train_examples))
  model.fit(
      train_images, train_labels,
      epochs=opts.train_epochs,
      # NOTE: steps_per_epoch will cause OOM for some reason.
      validation_data=dataset_eval,
      batch_size=opts.batch_size,
      callbacks=[tensorboard_cb],
  )
  return model

## MNIST

### Dataset

In [ ]:
def build_dataset(opts):
  """Returns an <images, labels> dataset pair."""
  opts = MnistDataOptions(**opts)
  logging.info('Building dataset with options: %s', opts)

  # We can't use in-distribution data from tfds due to inconsistent orderings.
  if opts.dataset_name == 'mnist':
    dataset = _mnist_dataset_from_tfr(opts.split)
  elif opts.dataset_name == 'not_mnist':
    dataset = _not_mnist_dataset_from_tfr(opts.split)
  else:
    dataset = _dataset_from_tfds(opts.dataset_name, opts.split)

  # Download dataset to memory.
  images, labels = list(zip(*tfds.as_numpy(dataset.batch(10**4))))
  images = np.concatenate(images, axis=0).astype(np.float32)
  labels = np.concatenate(labels, axis=0)

  images /= 255
  if opts.rotate_degs:
    images = scipy.ndimage.rotate(images, opts.rotate_degs, axes=[-2, -3])
    images = _crop_center(images, 28)
  if opts.roll_pixels:
    images = np.roll(images, opts.roll_pixels, axis=-2)

  return images, labels


### Hyper Parameters

In [8]:
_BATCH_SIZE_FOR_PREDICT = 100

HParams = collections.namedtuple(
    'Hparams', ['batch_size', 'learning_rate']
)


_HPS_DICT = {
    'mnist'=dict(vanilla=HParams(64, 0.0004), svi=HParams(2048, 0.003)),
    'cifar'=dict(vanilla=HParams(64, 0.0004), svi=HParams(2048, 0.003))}


def get_tuned_model_options(method):
  hps = _HPS_DICT[method]

  num_train_examples = (data.lib.NUM_TRAIN_EXAMPLES)
  model_opts = models_lib.ModelOptions(
      method=method,
      train_epochs=_TRAIN_EPOCHS,
      num_train_examples=num_train_examples,
      batch_size=hps.batch_size,
      learning_rate=hps.learning_rate,
      num_examples_for_predict=int(1e4),
      predictions_per_example=_PREDICTIONS_PER_EXAMPLE,
      input_shape=(28, 28),
      num_classes=10
  )

  if method == 'vanilla':
    model_opts.predictions_per_example = 1
  return model_opts

In [ ]:
gfile = tf.io.gfile

def get_experiment_config(method,
                          test_level, output_dir=None):
  """Returns model and data configs."""
  data_opts_list = data_lib.DATA_OPTIONS_LIST
  if test_level:
    data_opts_list = data_opts_list[:4]

  model_opts = hparams_lib.get_tuned_model_options(method)

  if output_dir:
    experiment_utils.record_config(model_opts, output_dir+'/model_options.json')
  return model_opts, data_opts_list


def run(method, output_dir, test_level):
  """Trains a model and records its predictions on configured datasets.

  Args:
    method: Name of modeling method (vanilla, svi).
    output_dir: Directory to record the trained model and output stats.
    test_level: Zero indicates no testing. One indicates testing with real data.
  """
  gfile.makedirs(output_dir)
  model_opts, data_opts_list = get_experiment_config(method,
                                                     test_level=test_level,
                                                     output_dir=output_dir)

  # Separately build dataset[0] with shuffle=True for training.
  dataset_train = data_lib.build_dataset(data_opts_list[0])
  dataset_eval = data_lib.build_dataset(data_opts_list[1])
  model = models_lib.build_and_train(model_opts,
                                     dataset_train, dataset_eval, output_dir)
  logging.info('Saving model to output_dir.')
  model.save_weights(output_dir + '/model.ckpt')

  for idx, data_opts in enumerate(data_opts_list):
    dataset = data_lib.build_dataset(data_opts)
    logging.info('Running predictions for dataset #%d', idx)
    stats = models_lib.make_predictions(model_opts, model, dataset)
    array_utils.write_npz(output_dir, 'stats_%d.npz' % idx, stats)
    del stats['logits_samples']
    array_utils.write_npz(output_dir, 'stats_small_%d.npz' % idx, stats)


## CIFAR-10